## PURPOSE OF THIS NOTEBOOK

The goal here is to test code and functions that will be used to automate the finding of key corners. 

In [1]:
import pandas as pd
import numpy as np
import fastf1
import fastf1.plotting
import matplotlib as mpl
import datetime
import timedelta

from scipy import stats
from scipy.signal import find_peaks
from sklearn.preprocessing import normalize
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

## Starting off 
I'm going to start off by doing a 1 lap experiement on testing code that points out which corners are key to analyizing driver braking confidence. Part of this will be filtering and ranking each of the predefined zones and only analyzing the most important zones. 

In [5]:
# Get lap information
race = fastf1.get_session(2025, 4, 'R')
race.load()

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

In [10]:
# Going to evaluate only the fastest lap as an example, will add other drivers soon 
fastest_lap = race.laps.pick_fastest()
car_data = fastest_lap.get_telemetry().add_distance()
lap_tel = car_data.copy()

In [15]:
# Now Calculate the brakezones 
lap_tel['BrakeZone'] = lap_tel[lap_tel['Brake']].index.to_series().diff().ne(1).cumsum()

# Drop rows where brake zone is NaN
lap_tel = lap_tel.dropna(subset=['BrakeZone'])

In [ ]:
# Now Calculate other features such as Decleration
lap_tel['DistanceBraked'] = lap_tel.groupby('BrakeZone')['Distance'].diff().fillna(0)
lap_tel['TotalDistanceBraked'] = lap_tel.groupby('BrakeZone')['DistanceBraked'].transform('sum')
# Now Calculate Average Speed at braking
lap_tel['AvgSpeedBraking'] = lap_tel.groupby('BrakeZone')['Speed'].transform('mean')
# Calculate braking times and total laptime in seconds
lap_tel['TotalTime'] = lap_tel['Time'].iloc[-1].total_seconds()
brake_start_dict = {}
brake_end_dict = {}
brake_start_speed_dict = {}
brake_end_speed_dict = {}
for zone, group in lap_tel.groupby('BrakeZone'):
    brake_start_dict[zone] = group['Time'].min().total_seconds()
    brake_end_dict[zone] = group['Time'].max().total_seconds()
    brake_start_speed_dict[zone] = group['Speed'].max()
    brake_end_speed_dict[zone] = group['Speed'].min()

# Map each new value 
lap_tel['SpeedatBrakeStart'] = lap_tel['BrakeZone'].map(brake_start_speed_dict)
lap_tel['SpeedatBrakeEnd'] = lap_tel['BrakeZone'].map(brake_end_speed_dict)
lap_tel['BrakeStartTime'] = lap_tel['BrakeZone'].map(brake_start_dict)
lap_tel['BrakeEndTime'] = lap_tel['BrakeZone'].map(brake_end_dict)
lap_tel['TimeBraking'] = lap_tel['BrakeEndTime'] - lap_tel['BrakeStartTime']

# Convert from kph to m/s
lap_tel['SpeedatBrakeStartMS'] = lap_tel['SpeedatBrakeStart'] * .2778
lap_tel['SpeedatBrakeEndMS'] = lap_tel['SpeedatBrakeEnd'] * .2778

# Calculate Deceleration
lap_tel['Deceleration'] = (lap_tel['SpeedatBrakeStartMS']) / lap_tel['TimeBraking']

/tmp/ipykernel_272437/2429687791.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lap_tel['DistanceBraked'] = lap_tel.groupby('BrakeZone')['Distance'].diff().fillna(0)
/tmp/ipykernel_272437/2429687791.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lap_tel['TotalDistanceBraked'] = lap_tel.groupby('BrakeZone')['DistanceBraked'].transform('sum')
/tmp/ipykernel_272437/2429687791.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [ ]:
# Now let's analyze braking zones 
def analyze_braking_zones(lap_tel):
    # Group by zones 
    grouped = lap_tel.groupby('BrakeZone')

In [23]:
lap_tel

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,...,AvgSpeedBraking,TotalTime,SpeedatBrakeStart,SpeedatBrakeEnd,BrakeStartTime,BrakeEndTime,TimeBraking,SpeedatBrakeStartMS,SpeedatBrakeEndMS,Deceleration
61,2025-04-13 16:04:20.969,0 days 01:56:06.702000,,NaN,0 days 00:00:07.329000,10015.000000,291.000000,8,0.0,True,...,144.731634,86.596,291.0,73.859999,7.329,10.437,3.108,80.8398,20.518308,26.010232
62,2025-04-13 16:04:21.117,0 days 01:56:06.850000,,NaN,0 days 00:00:07.477000,9240.114286,286.242857,8,0.0,True,...,144.731634,86.596,291.0,73.859999,7.329,10.437,3.108,80.8398,20.518308,26.010232
63,2025-04-13 16:04:21.249,0 days 01:56:06.982000,,NaN,0 days 00:00:07.609000,8549.000000,282.000000,8,0.0,True,...,144.731634,86.596,291.0,73.859999,7.329,10.437,3.108,80.8398,20.518308,26.010232
64,2025-04-13 16:04:21.417,0 days 01:56:07.150000,,NaN,0 days 00:00:07.777000,8503.865657,241.044763,8,0.0,True,...,144.731634,86.596,291.0,73.859999,7.329,10.437,3.108,80.8398,20.518308,26.010232
65,2025-04-13 16:04:21.450,0 days 01:56:07.183000,,NaN,0 days 00:00:07.810000,8495.000000,233.000000,8,0.0,True,...,144.731634,86.596,291.0,73.859999,7.329,10.437,3.108,80.8398,20.518308,26.010232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,2025-04-13 16:05:39.696,0 days 01:57:25.429000,,NaN,0 days 00:01:26.056000,10269.862209,152.724994,7,0.0,True,...,195.241887,86.596,291.0,133.628807,84.490,86.596,2.106,80.8398,37.122083,38.385470
646,2025-04-13 16:05:39.917,0 days 01:57:25.650000,,NaN,0 days 00:01:26.277000,9767.547771,142.595830,7,0.0,True,...,195.241887,86.596,291.0,133.628807,84.490,86.596,2.106,80.8398,37.122083,38.385470
647,2025-04-13 16:05:39.930,0 days 01:57:25.663000,,NaN,0 days 00:01:26.290000,9738.000000,142.000000,3,0.0,True,...,195.241887,86.596,291.0,133.628807,84.490,86.596,2.106,80.8398,37.122083,38.385470
648,2025-04-13 16:05:40.129,0 days 01:57:25.862000,,NaN,0 days 00:01:26.489000,9451.000000,136.000000,3,0.0,True,...,195.241887,86.596,291.0,133.628807,84.490,86.596,2.106,80.8398,37.122083,38.385470
